In [1]:
import os
import pandas as pd

print("🔷 Parquet 변환 시작")

# CSV 파일명과 출력 디렉토리 지정
CSV_FILE = 'combined_2024_all.csv'
OUTPUT_DIR = 'combined_parquet'

# 출력 디렉토리가 없으면 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# CSV를 chunk 단위로 읽기
chunksize = 10_000_000  # 1천만 행 단위
chunk_count = 0

for chunk in pd.read_csv(CSV_FILE, dtype=str, chunksize=chunksize):
    print(f"🔷 Chunk {chunk_count} 처리 중…")

    # etl_date 컬럼 생성
    chunk['ETL_DATE'] = pd.to_datetime(chunk['ETL_YMD'], format='%Y%m%d')

    # 파티션별로 나누어 저장
    # (여기선 chunk별 파일명을 다르게)
    chunk.to_parquet(
        f"{OUTPUT_DIR}/part_{chunk_count}.parquet",
        engine='pyarrow',
        index=False
    )

    print(f"✅ Chunk {chunk_count} Parquet 저장 완료")
    chunk_count += 1

print("🎉 전체 Parquet 변환 완료")


🔷 Parquet 변환 시작
🔷 Chunk 0 처리 중…
✅ Chunk 0 Parquet 저장 완료
🔷 Chunk 1 처리 중…
✅ Chunk 1 Parquet 저장 완료
🔷 Chunk 2 처리 중…
✅ Chunk 2 Parquet 저장 완료
🔷 Chunk 3 처리 중…
✅ Chunk 3 Parquet 저장 완료
🔷 Chunk 4 처리 중…
✅ Chunk 4 Parquet 저장 완료
🔷 Chunk 5 처리 중…
✅ Chunk 5 Parquet 저장 완료
🎉 전체 Parquet 변환 완료


In [ ]:
import polars as pl
import numpy as np

print("✅ 1️⃣ Parquet 스캔 시작")
lazy_df = pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
print("   → LazyFrame 생성 완료")

# 2️⃣ 날짜·시간·목적 필터  
filtered_step2 = (
    lazy_df
      .filter((pl.col('ETL_YMD') >= '20240101') & (pl.col('ETL_YMD') <= '20241231'))
      .filter((pl.col('TIME_CD').cast(pl.Int64) >= 10) & (pl.col('TIME_CD').cast(pl.Int64) <= 23))
      .filter(pl.col('MOVE_PURPOSE').is_in(['쇼핑','관광','출근','귀가','병원','기타']))
)

# 3️⃣ 주말 필터  
# 잘뽑힘

filtered_step3 = (
    filtered_step2
      .with_columns(pl.col('ETL_YMD').str.strptime(pl.Date, format='%Y%m%d').alias('ETL_DATE'))
      .filter(pl.col('ETL_DATE').dt.weekday().is_in([5, 6]))
)
# print("✅ 3️⃣ 주말 필터 후 샘플 5개:")
# print(filtered_step3.limit(5).collect())


# 4️⃣ 파생 컬럼 추가  
young_cols = ['MALE_20_CNT','MALE_30_CNT','FEML_20_CNT','FEML_30_CNT']
all_cols = young_cols + [ 
    'MALE_00_CNT','MALE_10_CNT','MALE_40_CNT','MALE_50_CNT','MALE_60_CNT','MALE_70_CNT',
    'FEML_00_CNT','FEML_10_CNT','FEML_40_CNT','FEML_50_CNT','FEML_60_CNT','FEML_70_CNT'
]

young_sum: pl.Expr = sum((pl.col(c).cast(pl.Float64) for c in young_cols), pl.lit(0.0))

total_sum: pl.Expr = sum((pl.col(c).cast(pl.Float64) for c in all_cols), pl.lit(0.0))

# purpose_flag = pl.when(pl.col("MOVE_PURPOSE").is_in(["쇼핑","관광"])).then(1.0).otherwise(0.0001)


# 쇼핑·관광 목적만 연령대별 방문자 수 합산
shopping_tour_sum: pl.Expr = sum(
    (pl.when(pl.col("MOVE_PURPOSE").is_in(["쇼핑", "관광"]))
      .then(pl.col(c).cast(pl.Float64))
      .otherwise(0.0)
      for c in all_cols
    ), pl.lit(0.0)
)



# 4-1️⃣ 먼저 YOUNG_VISITORS, TOTAL_VISITORS, PURPOSE_FLAG, YOUNG_RATIO 만 추가
step4a = filtered_step3.with_columns([
    young_sum.alias("YOUNG_VISITORS"),  #2030 합계
    total_sum.alias("TOTAL_VISITORS"),  #전체 합꼐
    
    shopping_tour_sum.alias("SHOPPING_TOUR_SUM"),
    
    # purpose_flag.alias("PURPOSE_FLAG"),
    # (shopping_tour_sum / (total_sum + 1e-6)).alias("SHOPPING_TOUR_RATIO")
      # 비율로 하면 분모인 total_sum 자체가 너무 커지는데..?
    
    (young_sum / (total_sum + 1e-6)).alias("YOUNG_RATIO"), #전체 중 2030 비율
])

# 아래 4-2부터 중요한데도 일단 step4a로 핫니스 계산 중이니까. 킵



# 4-2️⃣ 그다음에 HOTNESS 계산 (이제 YOUNG_RATIO 가 존재하므로 참조 가능)
filtered_step4 = step4a.with_columns([
    (
      pl.col("YOUNG_RATIO").pow(0.25)
      * pl.col("SHOPPING_TOUR_SUM")/pl.col("TOTAL_VISITORS").pow(0.25)
      # * pl.col("PURPOSE_FLAG").pow(0.5)
      
      * pl.col("TOTAL_VISITORS").log1p()
    ).alias("HOTNESS")
])

# # 분자는 1.
# # 0명인 지역이랑, 조정한 뒤의 관광쇼핑 비율이 최하위인 값을 대치하는 것.

# print("▶️ filtered_step4 컬럼:", filtered_step4.columns)

regions_to_sample = [
'서울특별시_마포구_합정동','서울특별시_성동구_성수1가2동', '서울특별시_송파구_잠실본동', '서울특별시_성동구_성수2가1동', '서울특별시_성동구_성수1가1동', '서울특별시_성동구_성수2가3동', '서울특별시_마포구_연남동', '서울특별시_송파구_잠실2동', '서울특별시_강남구_압구정동']

# print("✅ 4️⃣ 파생 컬럼 추가 후, 예상 핫플들 샘플 일별로 추출:")
# print(
#     filtered_step4
#       .filter(pl.col("FULL_NM").is_in(regions_to_sample))
#       .select(["ETL_DATE","YOUNG_VISITORS","TOTAL_VISITORS","SHOPPING_TOUR_SUM","YOUNG_RATIO","HOTNESS"])
#       .limit(5)
#       .collect()
# )

# ─── 4️⃣-1 연간 합산 샘플 추출 ────────────────────────────────────────────────
sample_agg = (
    filtered_step4
      # ① regions_to_sample 리스트에 든 동만 골라내고
      .filter(pl.col("FULL_NM").is_in(regions_to_sample))
      # ② FULL_NM(행정동)별로 묶어서
      .group_by("FULL_NM")
      # ③ HOTNESS를 1년치 모두 더한 TOTAL_HOTNESS_INDEX 컬럼 생성
      .agg([
          pl.sum("HOTNESS").alias("TOTAL_HOTNESS_INDEX"),
      ])
      # ④ 합계가 큰 순으로 내림차순 정렬
      .sort("TOTAL_HOTNESS_INDEX", descending=True)
      # ⑤ 실행
      .collect()
)

print("✅ 4️⃣-1 선택 지역 연간 합산 핫니스 지표:")
print(sample_agg)

✅ 1️⃣ Parquet 스캔 시작
   → LazyFrame 생성 완료


In [ ]:
# 배드타운 뽑기
  # 1. 서울시 전체 ‘구 단위’로 베드타운 지수 순위 필요

    # 1.a. 전체 구단위 
        #  - 동단위 탑 30

import polars as pl
import numpy as np
import pandas as pd

# 2️⃣ 날짜·시간·목적 필터 (평일 18~23시)
filtered = (
    lazy_df
      # 2024년 데이터만
      .filter((pl.col('ETL_YMD') >= '20240101') & (pl.col('ETL_YMD') <= '20241231'))
      # 18시(18) ~ 23시(23)
      .filter((pl.col('TIME_CD').cast(pl.Int64) >= 18)
            & (pl.col('TIME_CD').cast(pl.Int64) <= 23))
      # 전 목적군 중 “귀가”는 남겨두되 전체 방문자 집계용
      .filter(pl.col('MOVE_PURPOSE')
              .is_in(['쇼핑','관광','출근','귀가','병원','기타', '등교']))
      # ETL_DATE 컬럼을 Date 타입으로 바꿔
      .with_columns(
        pl.col('ETL_YMD')
          .str.strptime(pl.Date, format='%Y%m%d')
          .alias('ETL_DATE')
      )
      # 평일(월~금): weekday 0~4
      .filter(pl.col('ETL_DATE').dt.weekday().is_in([0,1,2,3,4]))
)

# 3️⃣ 동별 연간 ‘귀가’ vs ‘전체’ 방문자 집계
all_cols = [
    'MALE_00_CNT','MALE_10_CNT','MALE_20_CNT','MALE_30_CNT',
    'MALE_40_CNT','MALE_50_CNT','MALE_60_CNT','MALE_70_CNT',
    'FEML_00_CNT','FEML_10_CNT','FEML_20_CNT','FEML_30_CNT',
    'FEML_40_CNT','FEML_50_CNT','FEML_60_CNT','FEML_70_CNT'
]

# 3-1) ‘귀가’ 방문자 합
return_sum = sum(
    (
      pl.when(pl.col("MOVE_PURPOSE") == "귀가")
        .then(pl.col(c).cast(pl.Float64))
        .otherwise(0.0)
      for c in all_cols
    ),
    pl.lit(0.0)
).alias("SUM_RETURN")

# 3-2) 전체 방문자 합
total_sum = sum(
    (pl.col(c).cast(pl.Float64) for c in all_cols),
    pl.lit(0.0)
).alias("SUM_TOTAL")

agg = (
    filtered
    .with_columns([ return_sum, total_sum ])
    .group_by("FULL_NM")
    .agg([
      pl.col("SUM_RETURN").sum().alias("ANNUAL_RETURN"),
      pl.col("SUM_TOTAL").sum().alias("ANNUAL_TOTAL")
    ])
)

# 4️⃣ 비율 계산 및 순위
result = (
    agg
      .with_columns([
        # 귀가/전체 비율
        (pl.col("ANNUAL_RETURN") / (pl.col("ANNUAL_TOTAL") + 1e-6))
        .alias("RETURN_RATIO")
      ])
      # 높을수록 상위
      .sort("RETURN_RATIO", descending=True)
      # 순위 컬럼 추가 (1~)
      .with_row_count("RANK", offset=1)
      .limit(30)
      .select(["RANK","FULL_NM","ANNUAL_RETURN","ANNUAL_TOTAL","RETURN_RATIO"])
      .collect()
)

print("✅ 평일 18~23시 ‘귀가/전체’ 비율 TOP30:")
print(result)


/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_19384/3061769982.py:70: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)


✅ 평일 18~23시 ‘귀가/전체’ 비율 TOP30:
shape: (30, 5)
┌──────┬──────────────────────────────┬───────────────┬──────────────┬──────────────┐
│ RANK ┆ FULL_NM                      ┆ ANNUAL_RETURN ┆ ANNUAL_TOTAL ┆ RETURN_RATIO │
│ ---  ┆ ---                          ┆ ---           ┆ ---          ┆ ---          │
│ u32  ┆ str                          ┆ f64           ┆ f64          ┆ f64          │
╞══════╪══════════════════════════════╪═══════════════╪══════════════╪══════════════╡
│ 1    ┆ 서울특별시_관악구_난향동     ┆ 515458.28     ┆ 571720.17    ┆ 0.901592     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ 경기도_과천시_부림동         ┆ 572229.97     ┆ 646037.67    ┆ 0.885753     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ 경기도_군포시_오금동         ┆ 793592.32     ┆ 901053.87    ┆ 0.880738     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4    ┆ 서울특별시_관악구_청림동     ┆ 483709.96

In [ ]:
  # 1. 서울시 전체 ‘구 단위’로 베드타운 지수 순위 필요

    # 1.a. 전체 구단위 
        #  - 구단위 상위 정렬(서울만)

import polars as pl
import numpy as np
import pandas as pd


# 1️⃣ Parquet 스캔 & 서울특별시 필터
df = (
    pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
      .filter(pl.col("SIDO_NM") == "서울특별시")
)

# 2️⃣ 평일 18~23시, 이동목적 필터 & 날짜 파싱
df = (
    df
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      .filter((pl.col("TIME_CD").cast(pl.Int64) >= 18) & (pl.col("TIME_CD").cast(pl.Int64) <= 23))
      .filter(pl.col("MOVE_PURPOSE").is_in(
          ["쇼핑","관광","출근","귀가","병원","기타","등교"]
      ))
      .with_columns(
          pl.col("ETL_YMD").str.strptime(pl.Date, "%Y%m%d").alias("ETL_DATE")
      )
      .filter(pl.col("ETL_DATE").dt.weekday().is_in([0,1,2,3,4]))
)

# 3️⃣ 연령대별 방문자 합산을 위한 컬럼 리스트
all_age_cols = [
    "MALE_00_CNT","MALE_10_CNT","MALE_20_CNT","MALE_30_CNT",
    "MALE_40_CNT","MALE_50_CNT","MALE_60_CNT","MALE_70_CNT",
    "FEML_00_CNT","FEML_10_CNT","FEML_20_CNT","FEML_30_CNT",
    "FEML_40_CNT","FEML_50_CNT","FEML_60_CNT","FEML_70_CNT"
]

# 4️⃣ ‘귀가’ 합계 & 전체 합계 Expr 정의
return_sum = sum(
    (
      pl.when(pl.col("MOVE_PURPOSE") == "귀가")
        .then(pl.col(c).cast(pl.Float64))
        .otherwise(0.0)
      for c in all_age_cols
    ),
    pl.lit(0.0)
).alias("SUM_RETURN")

total_sum = sum(
    (pl.col(c).cast(pl.Float64) for c in all_age_cols),
    pl.lit(0.0)
).alias("SUM_TOTAL")

# 5️⃣ SGG_NM(자치구)별 집계
result = (
    df
      # ① 집계용 컬럼 추가
      .with_columns([return_sum, total_sum])
      # ② 구 단위 집계
      .group_by("SGG_NM")
      .agg([
        pl.col("SUM_RETURN").sum().alias("ANNUAL_RETURN"),
        pl.col("SUM_TOTAL").sum().alias("ANNUAL_TOTAL")
      ])
      # ③ 비율 계산
      .with_columns([
        (pl.col("ANNUAL_RETURN") / (pl.col("ANNUAL_TOTAL") + 1e-6))
          .alias("RETURN_RATIO")
      ])
      # ④ 내림차순 정렬 + 랭크 + 상위 30
      .sort("RETURN_RATIO", descending=True)
      .with_row_count("RANK", offset=1)
      .limit(30)
      # ⑤ 필요한 열만 선택
      .select(["RANK","SGG_NM","ANNUAL_RETURN","ANNUAL_TOTAL","RETURN_RATIO"])
      .collect()
)

print("✅ 자치구별 평일 18~23시 ‘귀가/전체’ 비율 TOP30:")
print(result)


/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_76761/797342007.py:72: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)


✅ 자치구별 평일 18~23시 ‘귀가/전체’ 비율 TOP30:
shape: (25, 5)
┌──────┬────────┬───────────────┬──────────────┬──────────────┐
│ RANK ┆ SGG_NM ┆ ANNUAL_RETURN ┆ ANNUAL_TOTAL ┆ RETURN_RATIO │
│ ---  ┆ ---    ┆ ---           ┆ ---          ┆ ---          │
│ u32  ┆ str    ┆ f64           ┆ f64          ┆ f64          │
╞══════╪════════╪═══════════════╪══════════════╪══════════════╡
│ 1    ┆ 양천구 ┆ 1.6009e7      ┆ 2.0431e7     ┆ 0.783544     │
│ 2    ┆ 은평구 ┆ 1.9432e7      ┆ 2.4895e7     ┆ 0.780562     │
│ 3    ┆ 도봉구 ┆ 1.1281e7      ┆ 1.4475e7     ┆ 0.779349     │
│ 4    ┆ 관악구 ┆ 2.0619e7      ┆ 2.6666e7     ┆ 0.773248     │
│ 5    ┆ 동작구 ┆ 1.5934e7      ┆ 2.0757e7     ┆ 0.767646     │
│ …    ┆ …      ┆ …             ┆ …            ┆ …            │
│ 21   ┆ 용산구 ┆ 9.0087e6      ┆ 1.3665e7     ┆ 0.659233     │
│ 22   ┆ 강남구 ┆ 2.4973e7      ┆ 3.8806e7     ┆ 0.64354      │
│ 23   ┆ 마포구 ┆ 1.5770e7      ┆ 2.4515e7     ┆ 0.643267     │
│ 24   ┆ 종로구 ┆ 6.1709e6      ┆ 1.1046e7     ┆ 0.558634     │
│ 25   ┆ 중구   ┆ 5

In [ ]:
# 전체 다보는 설정
import polars as pl

# ——— 생략 없는 전체 출력 설정 ———
pl.Config.set_tbl_rows(10_000)         # 최대 10,000행까지 표시
pl.Config.set_tbl_cols(1000)           # 최대 1,000열까지 표시
pl.Config.set_fmt_str_lengths(300)     # 셀당 최대 300문자까지 표시
pl.Config.set_tbl_formatting("UTF8_FULL")  # 전체 테이블을 풀 포맷으로

# (이제 나머지 코드 실행)
# 1️⃣ Parquet 스캔…
# …
# 5️⃣ 최종 result = … .collect()
print(result)  # 중간 생략 없이 전체가 보입니다.


shape: (25, 5)
┌──────┬──────────┬───────────────┬──────────────┬──────────────┐
│ RANK ┆ SGG_NM   ┆ ANNUAL_RETURN ┆ ANNUAL_TOTAL ┆ RETURN_RATIO │
│ ---  ┆ ---      ┆ ---           ┆ ---          ┆ ---          │
│ u32  ┆ str      ┆ f64           ┆ f64          ┆ f64          │
╞══════╪══════════╪═══════════════╪══════════════╪══════════════╡
│ 1    ┆ 양천구   ┆ 1.6009e7      ┆ 2.0431e7     ┆ 0.783544     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ 은평구   ┆ 1.9432e7      ┆ 2.4895e7     ┆ 0.780562     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ 도봉구   ┆ 1.1281e7      ┆ 1.4475e7     ┆ 0.779349     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4    ┆ 관악구   ┆ 2.0619e7      ┆ 2.6666e7     ┆ 0.773248     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5    ┆ 동작구   ┆ 1.5934e7      ┆ 2.0757e7     ┆ 0.767646     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6    ┆ 중

In [12]:
  # 1. 서울시 전체 ‘구 단위’로 베드타운 지수 순위 필요

    # 1.b. 송파구 내 동들 
import polars as pl
import numpy as np

print("✅ Parquet 스캔 시작 & 서울특별시·송파구 필터")
df = (
    pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
      .filter(pl.col("SIDO_NM") == "서울특별시")
      .filter(pl.col("SGG_NM")  == "송파구")
)

print("✅ 평일 18~23시·목적 필터 & 날짜 파싱")
df = (
    df
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      .filter((pl.col("TIME_CD").cast(pl.Int64) >= 18) & (pl.col("TIME_CD").cast(pl.Int64) <= 23))
      .filter(pl.col("MOVE_PURPOSE").is_in(
          ["쇼핑","관광","출근","귀가","병원","기타","등교"]
      ))
      .with_columns(
          pl.col("ETL_YMD").str.strptime(pl.Date, "%Y%m%d").alias("ETL_DATE")
      )
      .filter(pl.col("ETL_DATE").dt.weekday().is_in([0,1,2,3,4]))
)

all_age_cols = [
    "MALE_00_CNT","MALE_10_CNT","MALE_20_CNT","MALE_30_CNT",
    "MALE_40_CNT","MALE_50_CNT","MALE_60_CNT","MALE_70_CNT",
    "FEML_00_CNT","FEML_10_CNT","FEML_20_CNT","FEML_30_CNT",
    "FEML_40_CNT","FEML_50_CNT","FEML_60_CNT","FEML_70_CNT"
]

print("✅ ‘귀가’ vs 전체 방문자 합계 Expr 정의")
return_sum = sum(
    (
      pl.when(pl.col("MOVE_PURPOSE") == "귀가")
        .then(pl.col(c).cast(pl.Float64))
        .otherwise(0.0)
      for c in all_age_cols
    ),
    pl.lit(0.0)
).alias("SUM_RETURN")

total_sum = sum(
    (pl.col(c).cast(pl.Float64) for c in all_age_cols),
    pl.lit(0.0)
).alias("SUM_TOTAL")

print("✅ 동별 집계 & RETURN_RATIO 계산")
songpa_dong_rank = (
    df
      .with_columns([ return_sum, total_sum ])
      .group_by("FULL_NM")
      .agg([
        pl.col("SUM_RETURN").sum().alias("ANNUAL_RETURN"),
        pl.col("SUM_TOTAL").sum().alias("ANNUAL_TOTAL")
      ])
      .with_columns([
        (pl.col("ANNUAL_RETURN") / (pl.col("ANNUAL_TOTAL") + 1e-6))
          .alias("RETURN_RATIO")
      ])
      .sort("RETURN_RATIO", descending=True)
      .with_row_count("RANK", offset=1)
      .select(["RANK","FULL_NM","ANNUAL_RETURN","ANNUAL_TOTAL","RETURN_RATIO"])
      .collect()
)

print("✅ 송파구 내 동별 ‘귀가/전체’ 비율 TOP30:")
print(songpa_dong_rank)


✅ Parquet 스캔 시작 & 서울특별시·송파구 필터
✅ 평일 18~23시·목적 필터 & 날짜 파싱
✅ ‘귀가’ vs 전체 방문자 합계 Expr 정의
✅ 동별 집계 & RETURN_RATIO 계산


/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_76761/3759909243.py:65: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)


✅ 송파구 내 동별 ‘귀가/전체’ 비율 TOP30:
shape: (27, 5)
┌──────┬────────────────────────────┬───────────────┬──────────────┬──────────────┐
│ RANK ┆ FULL_NM                    ┆ ANNUAL_RETURN ┆ ANNUAL_TOTAL ┆ RETURN_RATIO │
│ ---  ┆ ---                        ┆ ---           ┆ ---          ┆ ---          │
│ u32  ┆ str                        ┆ f64           ┆ f64          ┆ f64          │
╞══════╪════════════════════════════╪═══════════════╪══════════════╪══════════════╡
│ 1    ┆ 서울특별시_송파구_잠실4동  ┆ 931620.37     ┆ 1.1019e6     ┆ 0.845475     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ 서울특별시_송파구_마천1동  ┆ 621065.51     ┆ 745579.32    ┆ 0.832997     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ 서울특별시_송파구_삼전동   ┆ 1.3260e6      ┆ 1.6195e6     ┆ 0.818728     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4    ┆ 서울특별시_송파구_송파2동  ┆ 814552.09     ┆ 999898.6     ┆ 0.81463

In [13]:
  # 1. 서울시 전체 ‘구 단위’로 베드타운 지수 순위 필요

    # 1.c. 관악구 내 동들 

        
import polars as pl
import numpy as np

print("✅ Parquet 스캔 시작 & 서울특별시·관악구 필터")
df = (
    pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
      .filter(pl.col("SIDO_NM") == "서울특별시")
      .filter(pl.col("SGG_NM")  == "관악구")
)

print("✅ 평일 18~23시·목적 필터 & 날짜 파싱")
df = (
    df
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      .filter((pl.col("TIME_CD").cast(pl.Int64) >= 18) & (pl.col("TIME_CD").cast(pl.Int64) <= 23))
      .filter(pl.col("MOVE_PURPOSE").is_in(
          ["쇼핑","관광","출근","귀가","병원","기타","등교"]
      ))
      .with_columns(
          pl.col("ETL_YMD").str.strptime(pl.Date, "%Y%m%d").alias("ETL_DATE")
      )
      .filter(pl.col("ETL_DATE").dt.weekday().is_in([0,1,2,3,4]))
)

all_age_cols = [
    "MALE_00_CNT","MALE_10_CNT","MALE_20_CNT","MALE_30_CNT",
    "MALE_40_CNT","MALE_50_CNT","MALE_60_CNT","MALE_70_CNT",
    "FEML_00_CNT","FEML_10_CNT","FEML_20_CNT","FEML_30_CNT",
    "FEML_40_CNT","FEML_50_CNT","FEML_60_CNT","FEML_70_CNT"
]

print("✅ ‘귀가’ vs 전체 방문자 합계 Expr 정의")
return_sum = sum(
    (
      pl.when(pl.col("MOVE_PURPOSE") == "귀가")
        .then(pl.col(c).cast(pl.Float64))
        .otherwise(0.0)
      for c in all_age_cols
    ),
    pl.lit(0.0)
).alias("SUM_RETURN")

total_sum = sum(
    (pl.col(c).cast(pl.Float64) for c in all_age_cols),
    pl.lit(0.0)
).alias("SUM_TOTAL")

print("✅ 동별 집계 & RETURN_RATIO 계산")
gwanak_dong_rank = (
    df
      .with_columns([ return_sum, total_sum ])
      .group_by("FULL_NM")
      .agg([
        pl.col("SUM_RETURN").sum().alias("ANNUAL_RETURN"),
        pl.col("SUM_TOTAL").sum().alias("ANNUAL_TOTAL")
      ])
      .with_columns([
        (pl.col("ANNUAL_RETURN") / (pl.col("ANNUAL_TOTAL") + 1e-6))
          .alias("RETURN_RATIO")
      ])
      .sort("RETURN_RATIO", descending=True)
      .with_row_count("RANK", offset=1)
      .select(["RANK","FULL_NM","ANNUAL_RETURN","ANNUAL_TOTAL","RETURN_RATIO"])
      .collect()
)

print("✅ 관악구 내 동별 ‘귀가/전체’ 비율 TOP30:")
print(gwanak_dong_rank)


✅ Parquet 스캔 시작 & 서울특별시·관악구 필터
✅ 평일 18~23시·목적 필터 & 날짜 파싱
✅ ‘귀가’ vs 전체 방문자 합계 Expr 정의
✅ 동별 집계 & RETURN_RATIO 계산


/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_76761/3385931839.py:67: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)


✅ 관악구 내 동별 ‘귀가/전체’ 비율 TOP30:
shape: (21, 5)
┌──────┬────────────────────────────┬───────────────┬──────────────┬──────────────┐
│ RANK ┆ FULL_NM                    ┆ ANNUAL_RETURN ┆ ANNUAL_TOTAL ┆ RETURN_RATIO │
│ ---  ┆ ---                        ┆ ---           ┆ ---          ┆ ---          │
│ u32  ┆ str                        ┆ f64           ┆ f64          ┆ f64          │
╞══════╪════════════════════════════╪═══════════════╪══════════════╪══════════════╡
│ 1    ┆ 서울특별시_관악구_난향동   ┆ 515458.28     ┆ 571720.17    ┆ 0.901592     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ 서울특별시_관악구_청림동   ┆ 483709.96     ┆ 551249.36    ┆ 0.877479     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ 서울특별시_관악구_행운동   ┆ 1.2981e6      ┆ 1.4825e6     ┆ 0.875616     │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4    ┆ 서울특별시_관악구_서림동   ┆ 978928.35     ┆ 1.1448e6     ┆ 0.8551 

In [16]:
import polars as pl

# 송파구 동별 RETURN_RATIO 기본 통계
songpa_stats = songpa_dong_rank.select([
    pl.col("RETURN_RATIO").mean().alias("mean"),
    pl.col("RETURN_RATIO").median().alias("median"),
    pl.col("RETURN_RATIO").min().alias("min"),
    pl.col("RETURN_RATIO").max().alias("max"),
    
    pl.col("RETURN_RATIO").quantile(0.25).alias("1분위수"),
    pl.col("RETURN_RATIO").quantile(0.75).alias("3분위수"),
])
print("▶️ 송파구 동별 귀가/전체 비율 통계:")
print(songpa_stats)

# 관악구 동별 RETURN_RATIO 기본 통계
gwanak_stats = gwanak_dong_rank.select([
    pl.col("RETURN_RATIO").mean().alias("mean"),
    pl.col("RETURN_RATIO").median().alias("median"),
    pl.col("RETURN_RATIO").min().alias("min"),
    pl.col("RETURN_RATIO").max().alias("max"),
    
    pl.col("RETURN_RATIO").quantile(0.25).alias("1분위수"),
    pl.col("RETURN_RATIO").quantile(0.75).alias("3분위수"),
])
print("\n▶️ 관악구 동별 귀가/전체 비율 통계:")
print(gwanak_stats)


▶️ 송파구 동별 귀가/전체 비율 통계:
shape: (1, 6)
┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ mean     ┆ median   ┆ min      ┆ max      ┆ 1분위수  ┆ 3분위수  │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 0.731343 ┆ 0.764245 ┆ 0.448127 ┆ 0.845475 ┆ 0.702108 ┆ 0.796645 │
└──────────┴──────────┴──────────┴──────────┴──────────┴──────────┘

▶️ 관악구 동별 귀가/전체 비율 통계:
shape: (1, 6)
┌─────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ mean    ┆ median   ┆ min      ┆ max      ┆ 1분위수  ┆ 3분위수  │
│ ---     ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ f64     ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞═════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 0.78865 ┆ 0.815932 ┆ 0.644346 ┆ 0.901592 ┆ 0.722017 ┆ 0.850378 │
└─────────┴──────────┴──────────┴──────────┴──────────┴────

In [ ]:

# ─── 2️⃣ 0이 아닌 최소 RATIO_SCORE 찾기 (Eager DataFrame) ────────────────────
# 위에서 collect() 한 ratio_sample 에선 부족하니,
# 전체 동을 모아 두었던 ratio_df 한 번만 collect 하거나
# 아래처럼 Lazy→collect 해도 됩니다.
ratio_df = (
    step4a
      .group_by("FULL_NM")
      .agg([
        pl.sum("SHOPPING_TOUR_SUM").alias("ANNUAL_SHOP_TOUR"),
        pl.sum("TOTAL_VISITORS").alias("ANNUAL_TOTAL_VISITORS"),
      ])
      .with_columns([
        (
          (pl.col("ANNUAL_SHOP_TOUR") / (pl.col("ANNUAL_TOTAL_VISITORS") + 1e-6))
          .pow(0.25)
        ).alias("RATIO_SCORE")
      ])
      .collect()
)

min_positive = (
    ratio_df
      .filter(pl.col("RATIO_SCORE") > 0)
      .select(pl.col("RATIO_SCORE").min())
      .to_series()[0]
)
print(f"2️⃣ RATIO_SCORE > 0 중 최소값: {min_positive:.6f}")

# ─── 2-1️⃣ RATIO_SCORE == 0 인 동 개수 확인 ─────────────────────────
zero_count = ratio_df.filter(pl.col("RATIO_SCORE") == 0).height
print(f"2-1️⃣ RATIO_SCORE == 0인 동 개수: {zero_count}")


# ─── 3️⃣ 0 보간 후 샘플 (상위 5행) ────────────────────────────────────────
filled_lf = (
    ratio_df.lazy()  # 다시 LazyFrame 으로 바꿔서
      .with_columns([
        pl.when(pl.col("RATIO_SCORE") == 0)
          .then(min_positive)
          .otherwise(pl.col("RATIO_SCORE"))
          .alias("RATIO_SCORE_FILLED")
      ])
)

filled_sample = (
    filled_lf
      .sort("RATIO_SCORE_FILLED", descending=False)  # 오름차순 정렬
      .limit(100)                                     # 하위 100개만
      .collect()
)

print("3️⃣ 0 보간 후 샘플 (하위 100):")
print(filled_sample)


# ─── 4️⃣ 최종 정렬 & 상위 30개 ───────────────────────────────────────────
result = (
    filled_lf
      .sort("RATIO_SCORE_FILLED", descending=True)  # 내림차순 정렬
      .limit(30)                                    # 상위 30개만
      .with_row_count("RANK", offset=1)             # 순위 컬럼 추가
      .select([
        "RANK",
        "FULL_NM",
        "ANNUAL_SHOP_TOUR",
        "ANNUAL_TOTAL_VISITORS",
        "RATIO_SCORE_FILLED"
      ])
      .collect()
)

print("✅ 최종 동별 연간 쇼핑·관광 기여도 TOP30:")
print(result)


1️⃣ 동별 연간 집계 및 RATIO_SCORE 샘플 (상위 5):
shape: (20, 4)
┌────────────────────────────────┬──────────────────┬───────────────────────┬─────────────┐
│ FULL_NM                        ┆ ANNUAL_SHOP_TOUR ┆ ANNUAL_TOTAL_VISITORS ┆ RATIO_SCORE │
│ ---                            ┆ ---              ┆ ---                   ┆ ---         │
│ str                            ┆ f64              ┆ f64                   ┆ f64         │
╞════════════════════════════════╪══════════════════╪═══════════════════════╪═════════════╡
│ 인천광역시_미추홀구_관교동     ┆ 668482.084       ┆ 1.8951e6              ┆ 0.770659    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_이천시_호법면           ┆ 459086.15        ┆ 1.4372e6              ┆ 0.751787    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_강서구_방화2동      ┆ 1.1289e6         ┆ 3.6016e6              ┆ 0.748241    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌

/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_19384/1381118892.py:81: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)             # 순위 컬럼 추가


In [26]:
# ─── 2-1️⃣ RATIO_SCORE == 0 인 동 개수 확인 ─────────────────────────
zero_count = ratio_df.filter(pl.col("RATIO_SCORE") == 0).height
print(f"2-1️⃣ RATIO_SCORE == 0인 동 개수: {zero_count}")


2-1️⃣ RATIO_SCORE == 0인 동 개수: 0


In [30]:


# 🔷  동별로 1년치 HOTNESS 를 합산
hotness_annual_top30 = (
    filtered_step4
      .group_by("FULL_NM")                        # 동별 집계
      .agg(pl.sum("HOTNESS").alias("ANNUAL_HOTNESS"))
      .sort("ANNUAL_HOTNESS", descending=True)       # 내림차순 정렬
      .with_row_count("RANK", offset=1)           # 1~30 순위
      .limit(30)                                  # 상위 30개 동
      .select(["RANK","FULL_NM","ANNUAL_HOTNESS"])
      .collect()
)

print("✅ 동별 연간 HOTNESS 합계 TOP30:")
print(hotness_annual_top30)


/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_19384/2790176775.py:7: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)           # 1~30 순위


✅ 동별 연간 HOTNESS 합계 TOP30:
shape: (30, 3)
┌──────┬────────────────────────────────┬────────────────┐
│ RANK ┆ FULL_NM                        ┆ ANNUAL_HOTNESS │
│ ---  ┆ ---                            ┆ ---            │
│ u32  ┆ str                            ┆ f64            │
╞══════╪════════════════════════════════╪════════════════╡
│ 1    ┆ 서울특별시_송파구_잠실3동      ┆ 1.6187e6       │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ 서울특별시_서초구_반포4동      ┆ 1.2323e6       │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ 서울특별시_강서구_방화2동      ┆ 1.0593e6       │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4    ┆ 경기도_성남시_분당구_백현동    ┆ 1.0506e6       │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5    ┆ 경기도_남양주시_다산1동        ┆ 870556.189957  │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6    ┆ 경기도_수원시_권선구_서둔동    ┆ 863631.267017  │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 7    ┆ 경기도_부천시_원미구_중1

In [ ]:
import polars as pl


# 🔷  동별로 1년치 HOTNESS 를 합산
hotness_annual_bottom300 = (
    filtered_step4
      .group_by("FULL_NM")                        # 동별 집계
      .agg(pl.sum("HOTNESS").alias("ANNUAL_HOTNESS"))
      .sort("ANNUAL_HOTNESS", descending=False)       # 내림차순 정렬
      .with_row_count("RANK", offset=1)           # 1~30 순위
      .limit(300)                                  # 상위 30개 동
      .select(["RANK","FULL_NM","ANNUAL_HOTNESS"])
      .collect()
)

print("✅ 동별 연간 HOTNESS 합계 TOP300:")
print(hotness_annual_bottom300)

/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_19384/1360059665.py:7: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)           # 1~30 순위


✅ 동별 연간 HOTNESS 합계 TOP300:
shape: (300, 3)
┌──────┬─────────────────────────────────┬────────────────┐
│ RANK ┆ FULL_NM                         ┆ ANNUAL_HOTNESS │
│ ---  ┆ ---                             ┆ ---            │
│ u32  ┆ str                             ┆ f64            │
╞══════╪═════════════════════════════════╪════════════════╡
│ 1    ┆ 서울특별시_양천구_신정4동       ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ 서울특별시_동작구_노량진2동     ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ 경기도_의정부시_신곡1동         ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4    ┆ 경기도_하남시_덕풍3동           ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5    ┆ 인천광역시_남동구_논현2동       ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6    ┆ 경기도_김포시_통진읍            ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤

In [38]:
# # (스크립트 맨 위에 한 번만 넣어 주시면 됩니다)
# # • 소수점 10자리까지 포맷
# pl.Config.set_fmt_float_format("{:.10f}")
# # • 생략 없는 전체 테이블 포맷
# pl.Config.set_tbl_formatting("UTF8_FULL")


# ───────────────────────────────────────────────────────────────
# 관심 지역 리스트
regions_to_sample = [
    '서울특별시_마포구_합정동',
    '서울특별시_성동구_성수1가2동',
    '서울특별시_송파구_잠실본동',
    '서울특별시_성동구_성수2가1동',
    '서울특별시_성동구_성수1가1동',
    '서울특별시_성동구_성수2가3동',
    '서울특별시_마포구_연남동',
    '서울특별시_송파구_잠실2동',
    '서울특별시_강남구_압구정동'
]


# ─── B) 연간 합산 HOTNESS ─────────────────────────────────────────
sample_annual = (
    filtered_step4
      .filter(pl.col("FULL_NM").is_in(regions_to_sample))  # 관심 지역만
      .group_by("FULL_NM")
      .agg(pl.sum("HOTNESS").alias("ANNUAL_HOTNESS"))      # 1년치 합계
      .sort("ANNUAL_HOTNESS", descending=True)                # 큰 순 정렬
      .with_row_count("RANK", offset=1)                    # 순위
      .collect()
)

print("▶️ 관심 지역 연간 HOTNESS 합계 순위:")
print(sample_annual)



import pandas as pd

# ① 결과를 Polars 에서 Eager DataFrame으로 만들고
df =sample_annual  # 이미 .collect() 된 LazyFrame이라고 가정

# ② Pandas 옵션 설정
pd.set_option("display.precision",    10)
pd.set_option("display.float_format", "{:.10f}".format)
pd.set_option("display.max_rows",     None)
pd.set_option("display.max_columns",  None)

# ③ Polars → Pandas 변환 후 print
print(df.to_pandas())

/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_19384/3234794611.py:30: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("RANK", offset=1)                    # 순위


▶️ 관심 지역 연간 HOTNESS 합계 순위:
shape: (9, 3)
┌──────┬──────────────────────────────┬────────────────┐
│ RANK ┆ FULL_NM                      ┆ ANNUAL_HOTNESS │
│ ---  ┆ ---                          ┆ ---            │
│ u32  ┆ str                          ┆ f64            │
╞══════╪══════════════════════════════╪════════════════╡
│ 1    ┆ 서울특별시_마포구_합정동     ┆ 1849.497043    │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ 서울특별시_성동구_성수2가3동 ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ 서울특별시_성동구_성수1가2동 ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4    ┆ 서울특별시_성동구_성수1가1동 ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5    ┆ 서울특별시_성동구_성수2가1동 ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6    ┆ 서울특별시_송파구_잠실2동    ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 7    ┆ 서울특별시_송파구_잠실본동   ┆ 0.0            │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# ------------------ 아래는 아직 보지말자

In [20]:
# 1️⃣ 동별 1년치 전체 방문자 합계 집계
dong_visitors = (
    step4a
      .group_by("FULL_NM")                                     # FULL_NM(행정동)별로 묶고
      .agg(
        pl.sum("TOTAL_VISITORS")                               # 각 동의 TOTAL_VISITORS를 모두 더해
          .alias("YEAR_TOTAL_VISITORS")                        # YEAR_TOTAL_VISITORS라는 이름으로 저장
      )
      .sort("YEAR_TOTAL_VISITORS", descending=True)               # 내림차순 정렬 (방문자 많은 순)
      .limit(20)                                               # 상위 20개 동만
      .collect()                                               # LazyFrame 실행
)

# 2️⃣ 결과 출력
print("✅ 동별 1년치 전체 방문자 수 TOP20:")
print(dong_visitors)


✅ 동별 1년치 전체 방문자 수 TOP20:
shape: (20, 2)
┌───────────────────────────────────┬─────────────────────┐
│ FULL_NM                           ┆ YEAR_TOTAL_VISITORS │
│ ---                               ┆ ---                 │
│ str                               ┆ f64                 │
╞═══════════════════════════════════╪═════════════════════╡
│ 서울특별시_종로구_종로1.2.3.4가동 ┆ 1.3370e7            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_영등포구_여의동        ┆ 1.3157e7            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_강남구_역삼1동         ┆ 1.2878e7            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_마포구_서교동          ┆ 1.2745e7            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 인천광역시_중구_운서동            ┆ 1.0703e7            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_남양주시_다산1동           ┆ 9.5844e6            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_

In [ ]:
# # 4-1️⃣ 먼저 YOUNG_VISITORS, TOTAL_VISITORS, PURPOSE_FLAG, YOUNG_RATIO 만 추가
# step4a = filtered_step3.with_columns([
#     young_sum.alias("YOUNG_VISITORS"),
#     total_sum.alias("TOTAL_VISITORS"),
#     purpose_flag.alias("PURPOSE_FLAG"),
#     (young_sum / (total_sum + 1e-6)).alias("YOUNG_RATIO"),
# ])

# print("▶️ step4a 컬럼:", step4a.columns)
# # 여기에서 "YOUNG_RATIO" 가 잘 생겼는지 확인해 보세요.

# # 4-2️⃣ 그다음에 HOTNESS 계산 (이제 YOUNG_RATIO 가 존재하므로 참조 가능)
# filtered_step4 = step4a.with_columns([
#     (
#       pl.col("YOUNG_RATIO").pow(0.25)
#       * pl.col("PURPOSE_FLAG").pow(0.5)
#       * pl.col("TOTAL_VISITORS").log1p()
#     ).alias("HOTNESS")
# ])

# print("▶️ filtered_step4 컬럼:", filtered_step4.columns)


▶️ step4a 컬럼: ['D_ADMDONG_CD', 'TIME_CD', 'MOVE_PURPOSE', 'MALE_00_CNT', 'MALE_10_CNT', 'MALE_20_CNT', 'MALE_30_CNT', 'MALE_40_CNT', 'MALE_50_CNT', 'MALE_60_CNT', 'MALE_70_CNT', 'FEML_00_CNT', 'FEML_10_CNT', 'FEML_20_CNT', 'FEML_30_CNT', 'FEML_40_CNT', 'FEML_50_CNT', 'FEML_60_CNT', 'FEML_70_CNT', 'TOTAL_CNT', 'ETL_YMD', 'ADMI_CD', 'SIDO_NM', 'SGG_NM', 'ADMI_NM', 'FULL_NM', 'ETL_DATE', 'YOUNG_VISITORS', 'TOTAL_VISITORS', 'PURPOSE_FLAG', 'YOUNG_RATIO']
▶️ filtered_step4 컬럼: ['D_ADMDONG_CD', 'TIME_CD', 'MOVE_PURPOSE', 'MALE_00_CNT', 'MALE_10_CNT', 'MALE_20_CNT', 'MALE_30_CNT', 'MALE_40_CNT', 'MALE_50_CNT', 'MALE_60_CNT', 'MALE_70_CNT', 'FEML_00_CNT', 'FEML_10_CNT', 'FEML_20_CNT', 'FEML_30_CNT', 'FEML_40_CNT', 'FEML_50_CNT', 'FEML_60_CNT', 'FEML_70_CNT', 'TOTAL_CNT', 'ETL_YMD', 'ADMI_CD', 'SIDO_NM', 'SGG_NM', 'ADMI_NM', 'FULL_NM', 'ETL_DATE', 'YOUNG_VISITORS', 'TOTAL_VISITORS', 'PURPOSE_FLAG', 'YOUNG_RATIO', 'HOTNESS']


/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_19384/4105146105.py:9: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print("▶️ step4a 컬럼:", step4a.columns)
/var/folders/34/nfsz0w6s5rl3lqtq7z6j5p_h0000gp/T/ipykernel_19384/4105146105.py:21: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print("▶️ filtered_step4 컬럼:", filtered_step4.columns)


In [1]:
from functools import reduce
import operator
import polars as pl

# (… 1,2 단계 생략: scan_parquet, 필터 …)

# 3️⃣ 파생 컬럼 정의
purpose_flag = (
    pl.when(pl.col("MOVE_PURPOSE").is_in(["쇼핑","관광"]))
      .then(1.0)
      .otherwise(0.0)
      .alias("PURPOSE_FLAG")
)

young_sum = reduce(operator.add,
    [pl.col(c).cast(pl.Float64) for c in young_cols]
).alias("YOUNG_VISITORS")

total_sum = reduce(operator.add,
    [pl.col(c).cast(pl.Float64) for c in all_cols]
).alias("TOTAL_VISITORS")

filtered_df = filtered_df.with_columns([
    purpose_flag,
    young_sum,
    total_sum,
    # 쇼핑/관광 방문자
    (pl.col("PURPOSE_FLAG") * pl.col("TOTAL_VISITORS"))
      .alias("SHOPPING_TOUR_VISITORS")
])

# 4️⃣ 비율, 루트, 로그 등 나머지 지표
filtered_df = filtered_df.with_columns([
    (pl.col("SHOPPING_TOUR_VISITORS") / (pl.col("TOTAL_VISITORS")+1e-6))
      .alias("SHOPPING_TOUR_RATIO"),
    (pl.col("YOUNG_VISITORS") / (pl.col("TOTAL_VISITORS")+1e-6))
      .alias("YOUNG_RATIO"),

    pl.col("YOUNG_RATIO").pow(0.25).alias("YOUNG_RATIO_4TH_ROOT"),
    pl.col("SHOPPING_TOUR_RATIO").pow(0.5).alias("SHOPPING_TOUR_RATIO_SQRT"),
    (pl.col("YOUNG_RATIO").pow(0.25) * pl.col("SHOPPING_TOUR_RATIO").pow(0.5))
      .alias("COMBINED_RATIO"),
    pl.col("TOTAL_VISITORS").log1p().alias("LOG_TOTAL_VISITORS"),
])

# (… 5,6 단계: target_region 필터 + collect + 출력 …)


NameError: name 'young_cols' is not defined

In [ ]:
# 쇼핑관광이 0인 애들 찾기

# ─── 2️⃣ 0이 아닌 최소 RATIO_SCORE 찾기 (Eager DataFrame) ────────────────────
# 위에서 collect() 한 ratio_sample 에선 부족하니,
# 전체 동을 모아 두었던 ratio_df 한 번만 collect 하거나
# 아래처럼 Lazy→collect 해도 됩니다.
ratio_df = (
    step4a
      .group_by("FULL_NM")
      .agg([
        pl.sum("SHOPPING_TOUR_SUM").alias("ANNUAL_SHOP_TOUR"),
        pl.sum("TOTAL_VISITORS").alias("ANNUAL_TOTAL_VISITORS"),
      ])
      .with_columns([
        (
          (pl.col("ANNUAL_SHOP_TOUR") / (pl.col("ANNUAL_TOTAL_VISITORS") + 1e-6))
          .pow(0.25)
        ).alias("RATIO_SCORE")
      ])
      .collect()
)

min_positive = (
    ratio_df
      .filter(pl.col("RATIO_SCORE") > 0)
      .select(pl.col("RATIO_SCORE").min())
      .to_series()[0]
)
print(f"2️⃣ RATIO_SCORE > 0 중 최소값: {min_positive:.6f}")

# ─── 2-1️⃣ RATIO_SCORE == 0 인 동 개수 확인 ─────────────────────────
zero_count = ratio_df.filter(pl.col("RATIO_SCORE") == 0).height
print(f"2-1️⃣ RATIO_SCORE == 0인 동 개수: {zero_count}")
